<a href="https://colab.research.google.com/github/anarghya15/notebooks/blob/main/notebooks/PartGLEE-Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install shapely==1.7.1
!pip install numpy==1.26.4
!pip install Pillow==8.4.0
!pip install lvis
!pip install scipy
!pip install fairscale
!pip install einops
!pip install xformers
!pip install tensorboard
!pip install opencv-python-headless
!pip install timm
!pip install ftfy
!pip install transformers==4.36.0

!pip install -e .
!pip install git+https://github.com/wjf5203/cocoapi.git#"egg=pycocotools&subdirectory=PythonAPI" --user

# Download pretrained Language Model (CLIP text encoder from transformers)
!wget -P projects/PartGLEE/clip_vit_base_patch32/ https://huggingface.co/spaces/Junfeng5/GLEE_demo/resolve/main/GLEE/clip_vit_base_patch32/pytorch_model.bin

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.2/383.2 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for shapely: filename=Shapely-1.7.1-py2.py3-none-any.whl size=298258 sha256=c7330b1f228383f9f189f421e87f1d8052344cc990ab03f8bd8170b656bc4a31
  Stored in directory: /root/.cache/pip/wheels/cb/c6/48/7e03d742a16fc4995e4b841cba776c2006fdf0fd6057be5681
Successfully built shapely
  Attempting uninstall: shapely
    Found existing installation: shapely 2.0.7
    Uninstalling shapely-2.0.7:
      Successfully uninstalled shapely-2.0.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
geopandas 1.0.1 requires shapely>=2.0.0, but you have shapely 1.7.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 MB 17.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for Pillow: filename=Pillo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.3/266.3 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━

In [1]:
!git clone https://github.com/ProvenceStar/PartGLEE.git

fatal: destination path 'PartGLEE' already exists and is not an empty directory.


In [2]:
# compile Deformable DETR
%cd projects/PartGLEE/partglee/models/pixel_decoder/ops/
!python setup.py build install --user

[Errno 2] No such file or directory: 'projects/PartGLEE/partglee/models/pixel_decoder/ops/'
/content
python3: can't open file '/content/setup.py': [Errno 2] No such file or directory


In [3]:
%cd /content/PartGLEE/projects/PartGLEE/
!mkdir checkpoint

/content/PartGLEE/projects/PartGLEE
mkdir: cannot create directory ‘checkpoint’: File exists


In [4]:
!pip install gdown
!gdown --id 1D4my_0SA1f0YR7JOpC-Op92rsiaPtgST -O /content/PartGLEE/projects/PartGLEE/checkpoint/PartGLEE_SwinL.pth

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1D4my_0SA1f0YR7JOpC-Op92rsiaPtgST
From (redirected): https://drive.google.com/uc?id=1D4my_0SA1f0YR7JOpC-Op92rsiaPtgST&confirm=t&uuid=bb2b2b87-bf4b-4c2e-a2b0-c09e66f050a8
To: /content/PartGLEE/projects/PartGLEE/checkpoint/PartGLEE_SwinL.pth
100% 1.52G/1.52G [00:27<00:00, 55.9MB/s]


In [ ]:
!wget http://images.cocodataset.org/zips/train2017.zip
!unzip train2017.zip -d /content/PartGLEE/datasets/coco/train2017
!rm train2017.zip

--2025-03-06 11:59:52--  http://images.cocodataset.org/zips/train2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.40.217, 3.5.25.190, 54.231.233.17, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.40.217|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19336861798 (18G) [application/zip]
Saving to: ‘train2017.zip.1’

train2017.zip.1      71%[=============>      ]  12.95G  16.9MB/s    eta 5m 20s 

In [ ]:
!wget http://images.cocodataset.org/zips/val2017.zip
!unzip val2017.zip -d /content/PartGLEE/datasets/coco/val2017
!rm val2017.zip

In [ ]:
!cd /content/PartGLEE

In [ ]:
!python projects/PartGLEE/train_net.py --config-file projects/PartGLEE/configs/Inference/Swin-L/coco.yaml  --eval-only